In [2]:
import os
import pandas as pd

# Ścieżka do folderu z plikami Excela
directory_path = r"D:\Python\ProjectsAndData\RKN"  # <- PODAJ SWOJĄ ŚCIEŻKĘ

# Słownik do ostatecznego ujednolicenia nazw
name_mapping = {
    "Data utworzenia dokumentu": "Data utworzenia dokumentu",
    "Koła Naukowego": "Koło Naukowe",
    "Nazwa Koła Naukowego": "Koło Naukowe",
    "Nazwa Organizacji Studenckiej/ Koła Naukowego": "Koło Naukowe",
    "Okres na który składany jest preliminarz": "Okres preliminarza",
    "Opiekun Koła Naukowego": "Opiekun Koła",
    "Opiekun Koła Naukowego (imię, nazwisko, katedra)": "Opiekun Koła",
    "Opiekunowie Koła Naukowego": "Opiekun Koła",
    "Osoba sporządzająca dokument, ze strony Koła": "Osoba sporządzająca dokument",
    "Osoba sporządzająca dokument, ze strony Koła (imię, nazwisko, funkcja,telefon, e-mail)": "Osoba sporządzająca dokument",
    "Osoba sporządzająca dokument, ze strony Organizacji/Koła": "Osoba sporządzająca dokument",
    "Osoba sporządzająca dokument, ze strony Organizacji/Koła (imię, nazwisko, funkcja,telefon, adres e-mail)": "Osoba sporządzająca dokument",
    "Przewodniczący Koła Naukowego": "Przewodniczący Koła",
    "Przewodniczący Koła Naukowego (imię i nazwisko)": "Przewodniczący Koła",
    "Przewodniczący Organizacji Studenckiej/Koła Naukowego": "Przewodniczący Koła",
    "Przewodniczący Organizacji Studenckiej/Koła Naukowego (imię i nazwisko)": "Przewodniczący Koła",
    "Przewodniczący Organizacji Studenckiej/Koła Naukowego (Imię i nazwisko)": "Przewodniczący Koła",
    "Przewodniczący Organizacji Studenckiej/Koła Naukowego (imię, nazwisko, katedra)": "Przewodniczący Koła",
    "Razem": "Razem",
    "Subwencja": "Subwencja"
}

# Lista do przechowywania wyników
data_list = []

# Iteracja po wszystkich plikach w folderze
for filename in os.listdir(directory_path):
    if filename.endswith(".xlsx") and not filename.startswith("~"):  # Pominięcie plików tymczasowych
        file_path = os.path.join(directory_path, filename)
        kolo_naukowe = None  # Przechowujemy wartość dla "Koło Naukowe"

        try:
            # Wczytanie pliku Excel
            xls = pd.ExcelFile(file_path)
            sheet_name = xls.sheet_names[0]  # Zakładamy, że dane są w pierwszym arkuszu
            df = pd.read_excel(xls, sheet_name=sheet_name, header=None)

            # Sprawdzenie liczby wierszy w arkuszu
            max_rows = df.shape[0]
            selected_indexes = [4, 5, 6, 7, 8, 9, 53]  # Pobieramy standardowe wiersze
            valid_indexes = [i for i in selected_indexes if i < max_rows]

            # Pobranie dostępnych wierszy (pierwsza kolumna)
            selected_rows = df.iloc[valid_indexes, 0]

            # Tworzenie listy wartości (każda zmienna w osobnej kolumnie)
            for row in selected_rows:
                if isinstance(row, str) and ":" in row:  # Wiersze zawierające nazwy zmiennych
                    key, value = row.split(":", 1)
                    key = key.strip()
                    value = value.strip()

                    # Ujednolicenie nazw zmiennych
                    key = name_mapping.get(key, key)  # Jeśli jest w słowniku, zamieniamy na ustandaryzowaną nazwę

                    # Jeśli to "Koło Naukowe", zapisujemy jego wartość
                    if key == "Koło Naukowe":
                        kolo_naukowe = value

                    data_list.append({
                        "Plik": filename.strip(),
                        "Nazwa zmiennej": key,
                        "Wartość": value
                    })

            # --- DODAJEMY POBIERANIE „RAZEM” ---
            # Sprawdzamy cały wiersz 53 dla słowa "Razem"
            if 53 in valid_indexes:
                razem_row = df.iloc[53, :].dropna().astype(str).tolist()  # Pobieramy wszystkie kolumny

                # Szukamy wartości liczbowej obok "Razem"
                for i, val in enumerate(razem_row):
                    if "Razem" in val:
                        # Sprawdzamy, czy kolejna kolumna zawiera liczbę
                        if i + 1 < len(razem_row) and razem_row[i + 1].replace('.', '', 1).isdigit():
                            data_list.append({
                                "Plik": filename.strip(),
                                "Nazwa zmiennej": "Razem",
                                "Wartość": razem_row[i + 1]
                            })
                        break  # Znaleźliśmy "Razem", nie musimy szukać dalej

        except Exception as e:
            print(f"Błąd w pliku {filename}: {e}")

# Konwersja listy słowników do DataFrame
result_df = pd.DataFrame(data_list)

# PODMIANA "Plik" na wartość z "Koło Naukowe"
for index, row in result_df.iterrows():
    if row["Nazwa zmiennej"] == "Koło Naukowe":
        # Podmieniamy wszystkie wiersze z danym plikiem na nazwę Koła Naukowego
        result_df.loc[result_df["Plik"] == row["Plik"], "Plik"] = row["Wartość"]

# Ścieżka do zapisu zbiorczego pliku wynikowego
output_file = os.path.join(directory_path, "Wyniki_Zbiorcze.xlsx")
result_df.to_excel(output_file, index=False)

print(f"Dane zostały zapisane w: {output_file}")


Dane zostały zapisane w: D:\Python\ProjectsAndData\RKN\Wyniki_Zbiorcze.xlsx
